# Week 5

# Lecture 9 - Speeding Up Learning (SGD, cross-entropy)

We already know what gradient descent is: we run our model on training data, compute the loss function, find the gradient of the loss function, and update our parameters in the opposite direction of the gradient by a small amount; and repeat this until the model converges. The same general approach is used in nearly all neural networks.

In linear regression and logistic classification, we were computing the outputs and loss function for the entire dataset (usually multiple times) per iteration of gradient descent, making a weight update based on the gradient, and repeating.

The typical approach used in modern neural networks is **stochastic** gradient descent (SGD). SGD updates weights after processing a random sample, called a **mini-batch**, of datapoints: enough points to reduce the variance of using just one for more stable convergence of parameters but not so much that the computation is too expensive. In this way, we process a mini-batch for outputs, compute an approximate loss function and approximate gradients, update weights, and repeat.

## How large should our mini-batches be?

Typically, using $2^n$ for something like $n=5, 6, 7, 8$ because these values tend to be ideal for the linear algebra optimization libraries we use (NumPy, TensorFlow, etc). The mini-batch size is a hyperparameter, but it generally isn't one you need to tweak too much.

One exception is if you are using GPUs for computing: then, it's typically best to choose the largest power of 2 that allows a whole mini-batch to fit into GPU memory, which will result in the underlying linear algebra libraries working optimally.

## Implementing SGD

Once we implement SGD with backpropagation, we will have constructed a "vanilla" neural network, which is probably the simplest neural network that is of practical use.

First, let's import some libraries we will be using.

In [8]:
# load some libraries
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from keras.datasets import mnist
from keras.utils import to_categorical

Next, we will write a class similar to the `FeedforwardNeuralNetwork` class we wrote previously but upgraded to use SGD.

In [9]:
class FeedforwardNeuralNetworkSGD:
    
    # input a vector [a, b, c, ...] with the number of nodes in each layer
    def __init__(self, layers, alpha = 0.1, batchSize = 32):
        # list of weight matrices between layers
        self.W = []
        
        # network architecture will be a vector of numbers of nodes for each layer
        self.layers = layers
        
        # learning rate
        self.alpha = alpha
        
        # batch size
        self.batchSize = batchSize
        
        # initialize the weights (randomly) -- this is our initial guess for gradient descent
        
        # initialize the weights between layers (up to the next-to-last one) as normal random variables
        for i in np.arange(0, len(layers) - 2):
            self.W.append(np.random.randn(layers[i] + 1, layers[i + 1] + 1))
            
        # initialize weights between the last two layers (we don't want bias for the last one)
        self.W.append(np.random.randn(layers[-2] + 1, layers[-1]))
        
    # define the sigmoid activation
    def sigmoid(self, x):
        return 1.0 / (1 + np.exp(-x))
    
    # define the sigmoid derivative (where z is the output of a sigmoid)
    def sigmoidDerivative(self, z):
        return z * (1 - z)
    
    # get a new mini-batch of data from the dataset
    def getNextBatch(self, X, y, batchSize):
        for i in np.arange(0, X.shape[0], batchSize):
            # yield returns a generator, which can continue where it left off on later calls
            # of the function
            yield (X[i:i + batchSize], y[i:i + batchSize])
    
    # fit the model
    def fit(self, X, y, epochs = 10000, update = 1000):
        # add a column of ones to the end of X
        X = np.hstack((X, np.ones([X.shape[0],1])))

        for epoch in range(epochs):
            
            # randomize the examples
            p = np.arange(0, X.shape[0])
            np.random.shuffle(p)
            X = X[p]
            y = y[p]

            # feed forward, backprop, and weight update
            for (x, target) in self.getNextBatch(X, y, self.batchSize):
                
                # make a list of output activations from the first layer
                # (just the original x values)
                A = [np.atleast_2d(x)]
                
                # feed forward
                for layer in range(len(self.W)):
                    
                    # feed through one layer and apply sigmoid activation
                    net = A[layer].dot(self.W[layer])
                    out = self.sigmoid(net)
                    
                    # add our network output to the list of activations
                    A.append(out)
                    
                # backpropagation (coming soon!)
                error = A[-1] - target
                
                D = [error * self.sigmoidDerivative(A[-1])]
                
                # loop backwards over the layers to build up deltas
                for layer in np.arange(len(A) - 2, 0, -1):
                    delta = D[-1].dot(self.W[layer].T)
                    delta = delta * self.sigmoidDerivative(A[layer])
                    D.append(delta)
                    
                # reverse the deltas since we looped in reverse
                D = D[::-1]
                
                # weight update in each layer
                for layer in range(len(self.W)):
                    self.W[layer] -= self.alpha * A[layer].T.dot(D[layer])
                    
            # print an 
            if (epoch + 1) % update == 0:
                loss = self.computeLoss(X,y)
                print('Epoch =', epoch + 1, '\t loss =', loss)
                
    def predict(self, X, addOnes = True):
        # initialize data, be sure it's the right dimension
        p = np.atleast_2d(X)
        
        # add a column of 1s for bias
        if addOnes:
            p = np.hstack((p, np.ones([X.shape[0],1])))
        
        # feed forward!
        for layer in np.arange(0, len(self.W)):
            p = self.sigmoid(np.dot(p, self.W[layer]))
            
        return p
    
    def computeLoss(self, X, y):
        # initialize data, be sure it's the right dimension
        y = np.atleast_2d(y)
        
        # feed the datapoints through the network to get predicted outputs
        predictions = self.predict(X, addOnes = False)
        loss = np.sum((predictions - y)**2) / 2.0
        
        return loss

## Example: MNIST

As promised, this SGD neural net should run faster, so let's try to use the full 60,000 training images available in MNIST and 10,000 test images. (This is still a LOT of computation, using 70000 total 28-by-28 images.

In [11]:
### CLASSIFY MNIST PICTURES

# load the full MNIST dataset: both data and labels
((trainX, trainY), (testX, testY)) = mnist.load_data()

# scale the data to values in [0,1]
trainX = trainX.astype('float32')/255.0
testX = testX.astype('float32')/255.0

# reshape the data
trainX = trainX.reshape([60000, 28*28])
testX = testX.reshape([10000, 28*28])

# convert the digits to one-hot vectors
trainY = to_categorical(trainY, 10)
testY = to_categorical(testY, 10)

# fit the model to the training data
model = FeedforwardNeuralNetworkSGD([784, 32, 16, 10], 0.5, 32)
model.fit(trainX, trainY, 100, 1)

# print the classification performance
print("Training set accuracy")
predictedY = model.predict(trainX)
predictedY = predictedY.argmax(axis=1)

trainY = trainY.argmax(axis=1)
print(classification_report(trainY, predictedY))

print("Test set accuracy")
predictedY = model.predict(testX)
predictedY = predictedY.argmax(axis=1)

testY = testY.argmax(axis=1)
print(classification_report(testY, predictedY))

Epoch = 1 	 loss = 7550.73396332593
Epoch = 2 	 loss = 6278.198063204192
Epoch = 3 	 loss = 6163.162225824945
Epoch = 4 	 loss = 5879.948644075109
Epoch = 5 	 loss = 5467.048761918217
Epoch = 6 	 loss = 5477.727315366954
Epoch = 7 	 loss = 5793.302567027704
Epoch = 8 	 loss = 5307.415873926209
Epoch = 9 	 loss = 2669.022852884081
Epoch = 10 	 loss = 2703.7245226453047
Epoch = 11 	 loss = 2779.666333005792
Epoch = 12 	 loss = 2366.296717810931
Epoch = 13 	 loss = 2309.327141786748
Epoch = 14 	 loss = 2301.109844891144
Epoch = 15 	 loss = 2628.4828140298514
Epoch = 16 	 loss = 2287.606308051975
Epoch = 17 	 loss = 2314.991463281257
Epoch = 18 	 loss = 2441.0273939919985
Epoch = 19 	 loss = 2017.6944442791453
Epoch = 20 	 loss = 2211.3749570749746
Epoch = 21 	 loss = 2079.1091248371995
Epoch = 22 	 loss = 1920.4527183639889
Epoch = 23 	 loss = 1840.7488678971313
Epoch = 24 	 loss = 1855.4309813523637
Epoch = 25 	 loss = 2083.1090949174527
Epoch = 26 	 loss = 1982.8011568145314
Epoch = 27 

Our test accuracy on MNIST jumped from mid-80\% previously to 95\% with our implementation using SGD and the full dataset!

### Cross-Entropy Loss Function

We will discuss in class, but the cross-entropy loss function can lead to faster training than the SSE we have used before, so we add it to the implementation.

In [14]:
class FeedforwardNeuralNetworkSGD:
    
    # input a vector [a, b, c, ...] with the number of nodes in each layer
    def __init__(self, layers, alpha = 0.1, batchSize = 32, loss = 'sum-of-squares'):
        # list of weight matrices between layers
        self.W = []
        
        # network architecture will be a vector of numbers of nodes for each layer
        self.layers = layers
        
        # learning rate
        self.alpha = alpha
        
        # batch size
        self.batchSize = batchSize
        
        # loss function
        self.loss = loss
        
        # initialize the weights (randomly) -- this is our initial guess for gradient descent
        
        # initialize the weights between layers (up to the next-to-last one) as normal random variables
        for i in np.arange(0, len(layers) - 2):
            self.W.append(np.random.randn(layers[i] + 1, layers[i + 1] + 1))
            
        # initialize weights between the last two layers (we don't want bias for the last one)
        self.W.append(np.random.randn(layers[-2] + 1, layers[-1]))
        
    # define the sigmoid activation
    def sigmoid(self, x):
        return 1.0 / (1 + np.exp(-x))
    
    # define the sigmoid derivative (where z is the output of a sigmoid)
    def sigmoidDerivative(self, z):
        return z * (1 - z)
    
    # get a new mini-batch of data from the dataset
    def getNextBatch(self, X, y, batchSize):
        for i in np.arange(0, X.shape[0], batchSize):
            # yield returns a generator, which can continue where it left off on later calls
            # of the function
            yield (X[i:i + batchSize], y[i:i + batchSize])
    
    # fit the model
    def fit(self, X, y, epochs = 10000, update = 1000):
        # add a column of ones to the end of X
        X = np.hstack((X, np.ones([X.shape[0],1])))

        for epoch in range(epochs):
            
            # randomize the examples
            p = np.arange(0, X.shape[0])
            np.random.shuffle(p)
            X = X[p]
            y = y[p]

            # feed forward, backprop, and weight update
            for (x, target) in self.getNextBatch(X, y, self.batchSize):
                
                # make a list of output activations from the first layer
                # (just the original x values)
                A = [np.atleast_2d(x)]
                
                # feed forward
                for layer in range(len(self.W)):
                    
                    # feed through one layer and apply sigmoid activation
                    net = A[layer].dot(self.W[layer])
                    out = self.sigmoid(net)
                    
                    # add our network output to the list of activations
                    A.append(out)
                    
                # backpropagation (coming soon!)
                error = A[-1] - target
                
                if self.loss == 'sum-of-squares':
                    D = [error * self.sigmoidDerivative(A[-1])]
                    
                if self.loss == 'cross-entropy':
                    D = [error]
                    
                # loop backwards over the layers to build up deltas
                for layer in np.arange(len(A) - 2, 0, -1):
                    delta = D[-1].dot(self.W[layer].T)
                    delta = delta * self.sigmoidDerivative(A[layer])
                    D.append(delta)
                    
                # reverse the deltas since we looped in reverse
                D = D[::-1]
                
                # weight update in each layer
                for layer in range(len(self.W)):
                    self.W[layer] -= self.alpha * A[layer].T.dot(D[layer])
                    
            # print an 
            if (epoch + 1) % update == 0:
                loss = self.computeLoss(X,y)
                print('Epoch =', epoch + 1, '\t loss =', loss)
                
    def predict(self, X, addOnes = True):
        # initialize data, be sure it's the right dimension
        p = np.atleast_2d(X)
        
        # add a column of 1s for bias
        if addOnes:
            p = np.hstack((p, np.ones([X.shape[0],1])))
        
        # feed forward!
        for layer in np.arange(0, len(self.W)):
            p = self.sigmoid(np.dot(p, self.W[layer]))
            
        return p
    
    def computeLoss(self, X, y):
        # initialize data, be sure it's the right dimension
        y = np.atleast_2d(y)
        
        # feed the datapoints through the network to get predicted outputs
        predictions = self.predict(X, addOnes = False)
        
        # if the loss function is sum of squares, compute it
        if self.loss == 'sum-of-squares':
            loss = np.sum((predictions - y)**2) / 2.0
            
        # if the loss function is cross-entropy, compute it
        if self.loss == 'cross-entropy':
            loss = np.sum(np.nan_to_num(-y * np.log(predictions) - (1 - y) * np.log(1 - predictions)))
        
        return loss

In [13]:
### CLASSIFY MNIST PICTURES

# load the full MNIST dataset: both data and labels
((trainX, trainY), (testX, testY)) = mnist.load_data()

# scale the data to values in [0,1]
trainX = trainX.astype('float32')/255.0
testX = testX.astype('float32')/255.0

# reshape the data
trainX = trainX.reshape([60000, 28*28])
testX = testX.reshape([10000, 28*28])

# convert the digits to one-hot vectors
trainY = to_categorical(trainY, 10)
testY = to_categorical(testY, 10)

# fit the model to the training data
model = FeedforwardNeuralNetworkSGD([784, 32, 16, 10], 0.1, 32, 'cross-entropy')
model.fit(trainX, trainY, 100, 1)

# print the classification performance
print("Training set accuracy")
predictedY = model.predict(trainX)
predictedY = predictedY.argmax(axis=1)

trainY = trainY.argmax(axis=1)
print(classification_report(trainY, predictedY))

print("Test set accuracy")
predictedY = model.predict(testX)
predictedY = predictedY.argmax(axis=1)

testY = testY.argmax(axis=1)
print(classification_report(testY, predictedY))

Epoch = 1 	 loss = 29630.10862089301
Epoch = 2 	 loss = 26798.07276149491
Epoch = 3 	 loss = 24379.503978421977
Epoch = 4 	 loss = 24079.809151101403
Epoch = 5 	 loss = 22375.06124818176
Epoch = 6 	 loss = 19919.297830755855
Epoch = 7 	 loss = 20495.219326776176
Epoch = 8 	 loss = 18769.072724242127
Epoch = 9 	 loss = 18462.009045060535
Epoch = 10 	 loss = 21170.264184095402
Epoch = 11 	 loss = 17836.531068153694
Epoch = 12 	 loss = 17044.358757381964
Epoch = 13 	 loss = 17122.634325310977
Epoch = 14 	 loss = 17063.5155182756
Epoch = 15 	 loss = 16846.952301319783
Epoch = 16 	 loss = 15730.22545906988
Epoch = 17 	 loss = 16525.977581555773
Epoch = 18 	 loss = 17543.388300024384
Epoch = 19 	 loss = 15793.178808596027
Epoch = 20 	 loss = 16232.112654995528
Epoch = 21 	 loss = 14438.828732931403
Epoch = 22 	 loss = 14815.494418406173
Epoch = 23 	 loss = 15961.865188235157
Epoch = 24 	 loss = 14289.314648277455
Epoch = 25 	 loss = 15711.880159694787
Epoch = 26 	 loss = 15003.51078107749
Ep

We get similar to slightly better results here.